In [1]:

import collections
import math


def _get_ngrams(segment, max_order):
    """Extracts all n-grams upto a given maximum order from an input segment.
    Args:
      segment: text segment from which n-grams will be extracted.
      max_order: maximum length in tokens of the n-grams returned by this
          methods.
    Returns:
      The Counter containing all n-grams upto max_order in segment
      with a count of how many times each n-gram occurred.
    """
    ngram_counts = collections.Counter()
    for order in range(1, max_order + 1):
        for i in range(0, len(segment) - order + 1):
            ngram = tuple(segment[i:i + order])
            ngram_counts[ngram] += 1
    return ngram_counts


def compute_bleu(reference_corpus, translation_corpus, max_order=4,
                 smooth=False):
    """Computes BLEU score of translated segments against one or more references.
    Args:
      reference_corpus: list of lists of references for each translation. Each
          reference should be tokenized into a list of tokens.
      translation_corpus: list of translations to score. Each translation
          should be tokenized into a list of tokens.
      max_order: Maximum n-gram order to use when computing BLEU score.
      smooth: Whether or not to apply Lin et al. 2004 smoothing.
    Returns:
      3-Tuple with the BLEU score, n-gram precisions, geometric mean of n-gram
      precisions and brevity penalty.
    """
    matches_by_order = [0] * max_order
    possible_matches_by_order = [0] * max_order
    reference_length = 0
    translation_length = 0
    for (references, translation) in zip(reference_corpus,
                                         translation_corpus):
        reference_length += min(len(r) for r in references)
        translation_length += len(translation)

        merged_ref_ngram_counts = collections.Counter()
        for reference in references:
            merged_ref_ngram_counts |= _get_ngrams(reference, max_order)
        translation_ngram_counts = _get_ngrams(translation, max_order)
        overlap = translation_ngram_counts & merged_ref_ngram_counts
        for ngram in overlap:
            matches_by_order[len(ngram) - 1] += overlap[ngram]
        for order in range(1, max_order + 1):
            possible_matches = len(translation) - order + 1
            if possible_matches > 0:
                possible_matches_by_order[order - 1] += possible_matches

    precisions = [0] * max_order
    for i in range(0, max_order):
        if smooth:
            precisions[i] = ((matches_by_order[i] + 1.) /
                             (possible_matches_by_order[i] + 1.))
        else:
            if possible_matches_by_order[i] > 0:
                precisions[i] = (float(matches_by_order[i]) /
                                 possible_matches_by_order[i])
            else:
                precisions[i] = 0.0

    if min(precisions) > 0:
        p_log_sum = sum((1. / max_order) * math.log(p) for p in precisions)
        geo_mean = math.exp(p_log_sum)
    else:
        geo_mean = 0

    ratio = (float(translation_length) / reference_length) if reference_length > 0 else 0.0

    if ratio > 1.0:
        bp = 1.
    else:
        bp = math.exp(1 - 1. / ratio) if ratio > 0 else 0.0

    bleu = geo_mean * bp

    return (bleu, precisions, bp, ratio, translation_length, reference_length)

In [2]:
import os 

from nltk import word_tokenize

In [25]:
pre_path = "../outputs_en_ru//"
path = pre_path + "20K/"

In [31]:
files = os.listdir(path)

In [32]:
files

['init_fuse.txt', 'init_no_fuse.txt', 'no_init_no_fuse.txt']

In [28]:
with open(pre_path + "output.txt") as f:
    ref_lines = f.read().splitlines()

In [29]:
ref = [[word_tokenize(x)] for x in ref_lines]

In [33]:
for f_name in files:
    with open(path + f_name) as f:
        target_lines = f.read().splitlines()
       
    target = [word_tokenize(x) for x in target_lines]
    
    print(f_name, compute_bleu(ref[:], target[:])[0])

init_fuse.txt 0.2715294747336381
init_no_fuse.txt 0.27727729881363755
no_init_no_fuse.txt 0.26330739221145333


In [19]:
for name, f_name in zip(names, files):
    with open("./" + f_name) as f:
        target_lines = f.read().splitlines()
       
    target = [word_tokenize(x) for x in target_lines]
    
    print(name, target_lines[0])

init_no_fuse Enfin , je voudrais remercier le commissaire d' avoir fait preuve de beaucoup d' ouverture à nos propositions .
fuse_no_init Enfin , je remercie M . le commissaire pour l' être très ouvert à nos propositions .
no_fuse_no_init Pour terminer , je voudrais remercier le commissaire d' être très ouvert à nos propositions .
fuse_init Enfin , je voudrais remercier le commissaire pour l' être très ouverte à nos propositions .
target Enfin , je voudrais saluer l ' excellente disposition dont la commissaire fait montre concernant nos propositions .


In [45]:
compute_bleu(ref[:], target[:])

(0.1408822747028321,
 [0.4967637540453074,
  0.22073578595317725,
  0.11072664359861592,
  0.05734767025089606],
 0.867278768135988,
 0.8753541076487252,
 618,
 706)

In [25]:
ref_lines[:5]

["Enfin , je voudrais saluer l ' excellente disposition dont la commissaire fait montre concernant nos propositions .",
 "Monsieur le Président , Madame la Commissaire , chers collègues , le chiffre catastrophique de 42 000 morts sur les routes de l ' Union européenne chaque année , les blessés , dont le nombre s ' élève à 1 , 7 million , nous incitent constamment à considérer le thème de la sécurité des transports comme un problème très important et prioritaire .",
 "Ces chiffres cachent non seulement une souffrance personnelle incommensurable , mais ils impliquent aussi d ' énormes pertes économiques , et donc d ' énormes coûts .",
 "Les estimations se chiffrent à 100 milliards d ' euros , ce qui correspond à environ 2 % du produit intérieur brut de tous les États membres européens .",
 "C ' est plus que ce que tous les États membres de l ' Union européenne réunis dépensent pour la culture ."]

In [26]:
target_lines[:5]

["Enfin , je remercie M . le commissaire pour l' être très ouvert à nos propositions .",
 "Monsieur le Président , Monsieur le Commissaire , chers collègues , le chiffre effroyable de 42 , 000 à un an sur les routes de l' Union européenne , on estime que 1 , 7 millions de livres , c' est - à - dire que nous devons accorder la priorité à la sécurité routière .",
 "Ces chiffres ne constituent pas seulement des grands critères , bien qu' ils constituent également des sujets économiques et de coûts .",
 "Ceux - ci estime qu' à 100 milliards d' euros , la raison pour certains 2 % du produit intérieur brut de l' ensemble des États membres européens .",
 "C' est plus que tous les États membres de l' Union européenne ont mis un terme à leur culture ."]

In [35]:
ref[0]

['Enfin',
 ',',
 'je',
 'voudrais',
 'saluer',
 'l',
 "'",
 'excellente',
 'disposition',
 'dont',
 'la',
 'commissaire',
 'fait',
 'montre',
 'concernant',
 'nos',
 'propositions',
 '.']